In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf

In [6]:
pip install pyupbit


     -------------------------------------- 125.0/125.0 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pyupbit

In [8]:
df = pyupbit.get_ohlcv(ticker="KRW-ETH", interval="day", count=365, period=1)

In [10]:
def makeX(df, i):
    tempX = []
    tempX.append(df['open'].iloc[i])
    tempX.append(df['high'].iloc[i])
    tempX.append(df['low'].iloc[i])
    tempX.append(df['close'].iloc[i])
    tempX.append(df['volume'].iloc[i])
    return tempX

# 결과를 저장할 리스트
x = []

# DataFrame의 행을 순회하며 makeX 함수 호출하여 x에 추가
for i in range(len(df)):
    x.append(makeX(df, i))

In [12]:
def makeY(df, i):
    # i+1이 유효한 인덱스 범위 내에 있는지 확인합니다.
    if i + 1 < len(df['close']):
        # 현재 인덱스의 다음 날과 현재 날의 종가 차이를 계산합니다.
        gap = df['close'].iloc[i+1] - df['close'].iloc[i]
        
        # 차이가 양수인 경우 1, 그렇지 않으면 0을 할당합니다.
        tempY = 1 if gap > 0 else 0
        return tempY
    else:
        # i+1이 범위를 벗어날 경우 처리합니다.
        return None  # 기본값을 반환하거나 다르게 처리할 수 있습니다.

# 예시 사용법
y = []

# 마지막 인덱스를 제외하고 범위를 반복하여 out-of-bounds 문제를 피합니다.
for i in range(len(df) - 1):
    y.append(makeY(df, i))


In [13]:
def scaleDown(df, name, i):
    return round(df[name].iloc[i]/math.pow(10,len(str(int(df[name].iloc[i])))),3)

In [14]:
def makeX(df, i):
    tempX = []
    tempX.append(scaleDown(df, 'open', i))
    tempX.append(scaleDown(df, 'high', i))
    tempX.append(scaleDown(df, 'low', i))
    tempX.append(scaleDown(df, 'close', i))
    tempX.append(scaleDown(df, 'volume', i))
    return tempX

In [15]:
#인풋 데이터 5개 이므로 shape=(5,)를 대입
inputs = tf.keras.Input(shape=(5,))

# h1 ~ h3은 히든 레이어, 층이 깊을 수록 정확도가 높아질 수 있음
# relu, tanh는 활성화 함수의 종류
h1 = tf.keras.layers.Dense(128, activation='relu')(inputs)
h2 = tf.keras.layers.Dense(128, activation='tanh')(h1)
h3 = tf.keras.layers.Dense(128, activation='relu')(h2)

# 값을 0 ~ 1 사이로 표현할 경우 sigmoid 활성화 함수 활용
# 마지막 아웃풋 값은 1개여야 함
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h3)

# 인풋, 아웃풋 설정을 대입하여 모델 생성 
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [16]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [20]:
import math

In [21]:
import numpy as np

In [26]:
# 인풋/아웃풋 데이터 생성
for i in range(len(df) - 2):  # 조건 변경
    x_sample = makeX(df, i)
    y_sample = makeY(df, i)
    
    if x_sample is not None and y_sample is not None:
        x.append(x_sample)
        y.append(y_sample)

# 데이터 크기 출력
print("x sizes:", len(x))
print("y sizes:", len(y))

# x와 y의 크기 맞추기
min_size = min(len(x), len(y))
x = x[:min_size]
y = y[:min_size]

# 인풋, 아웃풋 데이터를 numpy 포맷으로 대입
# epochs는 학습 반복 횟수
fitInfo = model.fit(np.array(x), np.array(y), epochs=2000)


x sizes: 2543
y sizes: 2542
Epoch 1/2000
80/80 [==============================] - 1s 727us/step - loss: 0.7028 - accuracy: 0.5094
Epoch 2/2000
80/80 [==============================] - 0s 633us/step - loss: 0.6958 - accuracy: 0.4953
Epoch 3/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6983 - accuracy: 0.5138
Epoch 4/2000
80/80 [==============================] - 0s 696us/step - loss: 0.7046 - accuracy: 0.5208
Epoch 5/2000
80/80 [==============================] - 0s 664us/step - loss: 0.6969 - accuracy: 0.5130
Epoch 6/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6953 - accuracy: 0.5303
Epoch 7/2000
80/80 [==============================] - 0s 734us/step - loss: 0.7016 - accuracy: 0.5087
Epoch 8/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6962 - accuracy: 0.5177
Epoch 9/2000
80/80 [==============================] - 0s 671us/step - loss: 0.6924 - accuracy: 0.5275
Epoch 10/2000
80/80 [==============================] -

80/80 [==============================] - 0s 620us/step - loss: 0.6643 - accuracy: 0.5763
Epoch 160/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6638 - accuracy: 0.5787
Epoch 161/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6652 - accuracy: 0.5755
Epoch 162/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6617 - accuracy: 0.5814
Epoch 163/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6653 - accuracy: 0.5696
Epoch 164/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6629 - accuracy: 0.5826
Epoch 165/2000
80/80 [==============================] - 0s 696us/step - loss: 0.6606 - accuracy: 0.5873
Epoch 166/2000
80/80 [==============================] - 0s 671us/step - loss: 0.6600 - accuracy: 0.5865
Epoch 167/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6623 - accuracy: 0.5838
Epoch 168/2000
80/80 [==============================] - 0s 663us/step - loss: 0

80/80 [==============================] - 0s 684us/step - loss: 0.6334 - accuracy: 0.5999
Epoch 316/2000
80/80 [==============================] - 0s 696us/step - loss: 0.6329 - accuracy: 0.6023
Epoch 317/2000
80/80 [==============================] - 0s 696us/step - loss: 0.6326 - accuracy: 0.6039
Epoch 318/2000
80/80 [==============================] - 0s 696us/step - loss: 0.6353 - accuracy: 0.5991
Epoch 319/2000
80/80 [==============================] - 0s 696us/step - loss: 0.6313 - accuracy: 0.6086
Epoch 320/2000
80/80 [==============================] - 0s 684us/step - loss: 0.6335 - accuracy: 0.6113
Epoch 321/2000
80/80 [==============================] - 0s 696us/step - loss: 0.6339 - accuracy: 0.6078
Epoch 322/2000
80/80 [==============================] - 0s 709us/step - loss: 0.6375 - accuracy: 0.6023
Epoch 323/2000
80/80 [==============================] - 0s 709us/step - loss: 0.6331 - accuracy: 0.5901
Epoch 324/2000
80/80 [==============================] - 0s 696us/step - loss: 0

80/80 [==============================] - 0s 646us/step - loss: 0.5922 - accuracy: 0.6400
Epoch 472/2000
80/80 [==============================] - 0s 646us/step - loss: 0.6040 - accuracy: 0.6381
Epoch 473/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6701 - accuracy: 0.5822
Epoch 474/2000
80/80 [==============================] - 0s 651us/step - loss: 0.6515 - accuracy: 0.5932
Epoch 475/2000
80/80 [==============================] - 0s 658us/step - loss: 0.6085 - accuracy: 0.6259
Epoch 476/2000
80/80 [==============================] - 0s 633us/step - loss: 0.5969 - accuracy: 0.6385
Epoch 477/2000
80/80 [==============================] - 0s 696us/step - loss: 0.5976 - accuracy: 0.6471
Epoch 478/2000
80/80 [==============================] - 0s 684us/step - loss: 0.5951 - accuracy: 0.6456
Epoch 479/2000
80/80 [==============================] - 0s 675us/step - loss: 0.5923 - accuracy: 0.6499
Epoch 480/2000
80/80 [==============================] - 0s 658us/step - loss: 0

80/80 [==============================] - 0s 684us/step - loss: 0.5503 - accuracy: 0.6692
Epoch 628/2000
80/80 [==============================] - 0s 652us/step - loss: 0.5577 - accuracy: 0.6629
Epoch 629/2000
80/80 [==============================] - 0s 671us/step - loss: 0.5538 - accuracy: 0.6707
Epoch 630/2000
80/80 [==============================] - 0s 671us/step - loss: 0.5489 - accuracy: 0.6747
Epoch 631/2000
80/80 [==============================] - 0s 684us/step - loss: 0.5447 - accuracy: 0.6806
Epoch 632/2000
80/80 [==============================] - 0s 658us/step - loss: 0.5629 - accuracy: 0.6751
Epoch 633/2000
80/80 [==============================] - 0s 658us/step - loss: 0.5530 - accuracy: 0.6766
Epoch 634/2000
80/80 [==============================] - 0s 658us/step - loss: 0.5661 - accuracy: 0.6715
Epoch 635/2000
80/80 [==============================] - 0s 671us/step - loss: 0.5545 - accuracy: 0.6633
Epoch 636/2000
80/80 [==============================] - 0s 671us/step - loss: 0

...

80/80 [==============================] - 0s 658us/step - loss: 0.4524 - accuracy: 0.7380
Epoch 1252/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4613 - accuracy: 0.7419
Epoch 1253/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4573 - accuracy: 0.7349
Epoch 1254/2000
80/80 [==============================] - 0s 707us/step - loss: 0.4592 - accuracy: 0.7423
Epoch 1255/2000
80/80 [==============================] - 0s 646us/step - loss: 0.4614 - accuracy: 0.7341
Epoch 1256/2000
80/80 [==============================] - 0s 696us/step - loss: 0.4578 - accuracy: 0.7388
Epoch 1257/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4640 - accuracy: 0.7337
Epoch 1258/2000
80/80 [==============================] - 0s 669us/step - loss: 0.4649 - accuracy: 0.7364
Epoch 1259/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4841 - accuracy: 0.7215
Epoch 1260/2000
80/80 [==============================] - 0s 658us/step 

80/80 [==============================] - 0s 658us/step - loss: 0.4629 - accuracy: 0.7372
Epoch 1408/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4426 - accuracy: 0.7522
Epoch 1409/2000
80/80 [==============================] - 0s 734us/step - loss: 0.4422 - accuracy: 0.7423
Epoch 1410/2000
80/80 [==============================] - 0s 684us/step - loss: 0.4431 - accuracy: 0.7455
Epoch 1411/2000
80/80 [==============================] - 0s 684us/step - loss: 0.4409 - accuracy: 0.7526
Epoch 1412/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4353 - accuracy: 0.7541
Epoch 1413/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4392 - accuracy: 0.7443
Epoch 1414/2000
80/80 [==============================] - 0s 722us/step - loss: 0.4556 - accuracy: 0.7360
Epoch 1415/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4770 - accuracy: 0.7246
Epoch 1416/2000
80/80 [==============================] - 0s 671us/step 

80/80 [==============================] - 0s 658us/step - loss: 0.4187 - accuracy: 0.7644
Epoch 1564/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4293 - accuracy: 0.7596
Epoch 1565/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4325 - accuracy: 0.7498
Epoch 1566/2000
80/80 [==============================] - 0s 633us/step - loss: 0.4261 - accuracy: 0.7640
Epoch 1567/2000
80/80 [==============================] - 0s 646us/step - loss: 0.4263 - accuracy: 0.7589
Epoch 1568/2000
80/80 [==============================] - 0s 646us/step - loss: 0.4265 - accuracy: 0.7541
Epoch 1569/2000
80/80 [==============================] - 0s 684us/step - loss: 0.4363 - accuracy: 0.7596
Epoch 1570/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4693 - accuracy: 0.7349
Epoch 1571/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4232 - accuracy: 0.7691
Epoch 1572/2000
80/80 [==============================] - 0s 671us/step 

80/80 [==============================] - 0s 646us/step - loss: 0.4352 - accuracy: 0.7592
Epoch 1720/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4362 - accuracy: 0.7640
Epoch 1721/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4722 - accuracy: 0.7293
Epoch 1722/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4243 - accuracy: 0.7573
Epoch 1723/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4404 - accuracy: 0.7577
Epoch 1724/2000
80/80 [==============================] - 0s 655us/step - loss: 0.4529 - accuracy: 0.7459
Epoch 1725/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4120 - accuracy: 0.7773
Epoch 1726/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4113 - accuracy: 0.7718
Epoch 1727/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4253 - accuracy: 0.7655
Epoch 1728/2000
80/80 [==============================] - 0s 658us/step 

80/80 [==============================] - 0s 658us/step - loss: 0.4319 - accuracy: 0.7620
Epoch 1876/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4198 - accuracy: 0.7655
Epoch 1877/2000
80/80 [==============================] - 0s 680us/step - loss: 0.4279 - accuracy: 0.7675
Epoch 1878/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4355 - accuracy: 0.7557
Epoch 1879/2000
80/80 [==============================] - 0s 658us/step - loss: 0.4325 - accuracy: 0.7624
Epoch 1880/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4034 - accuracy: 0.7773
Epoch 1881/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4161 - accuracy: 0.7667
Epoch 1882/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4131 - accuracy: 0.7758
Epoch 1883/2000
80/80 [==============================] - 0s 671us/step - loss: 0.4145 - accuracy: 0.7746
Epoch 1884/2000
80/80 [==============================] - 0s 671us/step 

In [27]:
result = {'accuracy': round(fitInfo.history['accuracy'][-1],2),
          'predict': round(model.predict([makeX(df, -2)])[0][0],2)}

print(result)

1/1 [==============================] - 0s 70ms/step
{'accuracy': 0.78, 'predict': 0.53}
